In [1]:
import os
import re
from pathlib import Path
from typing import Any, Dict
from collections import OrderedDict
import json
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
sns.set_theme(style="whitegrid")
%matplotlib agg

In [2]:
requiredStats = {
    # --- Performance Metrics ---
    'simSeconds': r'simSeconds\s+([0-9.]+)',
    'ipc': r'board\.processor\.cores\.core\.ipc\s+([0-9.]+)',
    'cpi': r'board\.processor\.cores\.core\.cpi\s+([0-9.]+)',
    'numInstructions': r'simInsts\s+([0-9]+)',
    'numCycles': r'board\.processor\.cores\.core\.numCycles\s+([0-9]+)',

    # --- Speculation and Pipeline Flush Statistics ---
    'branchLookups': r'board\.processor\.cores\.core\.branchPred\.lookups\s+([0-9]+)',
    'branchMispredicted': r'board\.processor\.cores\.core\.commit\.branchMispredicts\s+([0-9]+)',
    'commitSquashedInsts': r'board\.processor\.cores\.core\.commit\.commitSquashedInsts\s+([0-9]+)',
    
    # --- Memory Dependence Speculation Failures ---
    'memOrderViolations': r'board\.processor\.cores\.core\.lsq0\.memOrderViolation\s+([0-9]+)',

    # --- Cache and Memory Accesses (Energy & MPKI Proxies) ---
    'l1iAccesses': r'board\.cache_hierarchy\.l1icaches\.overallAccesses::total\s+([0-9]+)',
    'l1dAccesses': r'board\.cache_hierarchy\.l1dcaches\.overallAccesses::total\s+([0-9]+)',
    'l2Accesses': r'board\.cache_hierarchy\.l2cache\.overallAccesses::total\s+([0-9]+)',
    'dramReadAccesses': r'board\.memory\.mem_ctrl\.readReqs\s+([0-9]+)',
    'dramWriteAccesses': r'board\.memory\.mem_ctrl\.writeReqs\s+([0-9]+)',

    # --- Cache Misses (for Hit Rate & MPKI) ---
    'l1iMisses': r'board\.cache_hierarchy\.l1icaches\.overallMisses::total\s+([0-9]+)',
    'l1dHits': r'board\.cache_hierarchy\.l1dcaches\.overallHits::total\s+([0-9]+)',
    'l2Misses': r'board\.cache_hierarchy\.l2cache\.overallMisses::total\s+([0-9]+)',

    # --- Average Miss Latency (for AMAT) ---
    'l1iAvgMissLatency': r'board\.cache_hierarchy\.l1icaches\.overallAvgMissLatency::total\s+([0-9.]+)',
    'l2AvgMissLatency': r'board\.cache_hierarchy\.l2cache\.overallAvgMissLatency::total\s+([0-9.]+)',
    'memAvgLat': r'board\.memory\.mem_ctrl\.requestorReadAvgLat::processor\.cores\.core\.inst\s+([0-9.]+)',

    # --- Effective Stall Cycles ---
    'icacheStallCycles': r'board\.processor\.cores\.core\.fetchStats0\.icacheStallCycles\s+([0-9]+)',
    'dispatchRenameStalls': r'board\.processor\.cores\.core\.rename\.blockCycles\s+([0-9]+)',
    'dispatchIssueStalls': r'board\.processor\.cores\.core\.iew\.blockCycles\s+([0-9]+)',
    'iqFullEvents': r'board\.processor\.cores\.core\.iew\.iqFullEvents\s+([0-9]+)',
    
    # --- DRAM Energy and Power ---
    'dramTotalEnergy': r'board\.memory\.mem_ctrl\.dram\.rank0\.totalEnergy\s+([0-9]+)',
    'dramAveragePower': r'board\.memory\.mem_ctrl\.dram\.rank0\.averagePower\s+([0-9.]+)',
}

In [3]:
def collectStats(outputDir):
    data = []
    for root, dirs, files in os.walk(outputDir):
        if 'stats.txt' in files:
            statsFilePath = Path(root) / 'stats.txt'
            try:
                parameter = statsFilePath.parent.name
                kernel = statsFilePath.parent.parent.name
                
                currentStats: Dict[str, Any] = {
                    'parameter': parameter,
                    'kernel': kernel
                }

                with open(statsFilePath, 'r') as f:
                    content = f.read()
                    
                if content.strip() == "":
                    print(f"Stats file {statsFilePath} is empty. Skipping.")
                    continue

                for key, pattern in requiredStats.items():
                    match = re.search(pattern, content)
                    if match:
                        currentStats[key] = float(match.group(1))
                    else:
                        currentStats[key] = np.nan
                
                data.append(currentStats)
                print(f"  Successfully parsed: {kernel}/{parameter}")

            except (ValueError, IndexError) as e:
                print(f"Could not parse path structure for {statsFilePath}. Skipping. Error: {e}")
            except Exception as e:
                print(f"An unexpected error occurred while processing {statsFilePath}: {e}")
    return data

In [4]:
scriptDir = Path.cwd()
outputDir = scriptDir.parent / 'output'
data = collectStats(outputDir)

  Successfully parsed: compute/baseline
  Successfully parsed: compute/l1a_2
  Successfully parsed: compute/l1a_8
  Successfully parsed: compute/l1p_stride
  Successfully parsed: compute/l1p_tagged
  Successfully parsed: compute/l1rp_lru
  Successfully parsed: compute/l1rp_plru
  Successfully parsed: compute/l1rp_random
  Successfully parsed: compute/l1s_16KiB
  Successfully parsed: compute/l1s_64KiB
  Successfully parsed: compute/l2a_16
  Successfully parsed: compute/l2a_32
  Successfully parsed: compute/l2p_ampm
  Successfully parsed: compute/l2p_signature
  Successfully parsed: compute/l2p_stride
  Successfully parsed: compute/l2p_tagged
  Successfully parsed: compute/l2s_1MB
  Successfully parsed: compute/l2s_4MB
  Successfully parsed: compute/pred_bimodal
  Successfully parsed: compute/pred_local
  Successfully parsed: compute/pred_none
  Successfully parsed: compute/pred_perceptron
  Successfully parsed: compute/pred_tage
  Successfully parsed: compute/pw_12
  Successfully parsed

In [5]:
df = pd.DataFrame(data)
df

,parameter,kernel,simSeconds,ipc,cpi,numInstructions,numCycles,branchLookups,branchMispredicted,commitSquashedInsts,...,l2Misses,l1iAvgMissLatency,l2AvgMissLatency,memAvgLat,icacheStallCycles,dispatchRenameStalls,dispatchIssueStalls,iqFullEvents,dramTotalEnergy,dramAveragePower
0,baseline,compute,0.004596,2.285297,0.437580,33613812.0,14682751.0,1048734.0,3.0,3685.0,...,1.0,74337.500000,141476.000000,20208.00,100.0,340.0,340.0,0.0,2.140462e+09,465.753000
1,l1a_2,compute,0.004596,2.285297,0.437580,33613812.0,14682750.0,1048734.0,3.0,3678.0,...,1.0,74337.500000,141476.000000,20208.00,100.0,340.0,340.0,0.0,2.140466e+09,465.753971
2,l1a_8,compute,0.004596,2.285297,0.437580,33613812.0,14682750.0,1048734.0,3.0,3678.0,...,1.0,74337.500000,141476.000000,20208.00,100.0,340.0,340.0,0.0,2.140463e+09,465.753345
3,l1p_stride,compute,0.004596,2.285297,0.437580,33613812.0,14682751.0,1048734.0,3.0,3685.0,...,1.0,74337.500000,141476.000000,20208.00,100.0,340.0,340.0,0.0,2.140462e+09,465.753000
4,l1p_tagged,compute,0.004596,2.285317,0.437576,33613812.0,14682618.0,1048735.0,3.0,3685.0,...,NaN,102664.000000,NaN,18750.00,90.0,340.0,340.0,0.0,2.138983e+09,465.435439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,pw_12,stream,0.020655,1.366523,0.731784,112260013.0,65990491.0,9444359.0,26.0,46425.0,...,198985.0,143980.000000,94691.321798,64992.00,847.0,13586327.0,5727.0,0.0,1.291047e+10,625.052254
80,pw_8,stream,0.020578,1.371651,0.729048,112260013.0,65743781.0,9443149.0,1.0,52684.0,...,197044.0,75641.666667,93797.760566,47708.00,1027.0,446339.0,6355.0,0.0,1.287461e+10,625.655272
81,rob_128,stream,0.021299,1.325208,0.754598,112260013.0,68047837.0,9440997.0,2.0,32973.0,...,201225.0,91020.400000,95041.962256,62278.00,1151.0,480308.0,3891.0,0.0,1.321277e+10,620.347678
82,rob_256,stream,0.021143,1.334999,0.749064,112260013.0,67548795.0,9440930.0,1.0,32708.0,...,199698.0,118783.500000,94289.252061,68944.67,1186.0,242442.0,3818.0,0.0,1.314141e+10,621.555681


In [6]:
CLK_FREQ_GHZ = 3.2
L1_HIT_TIME_CYCLES = 1
L2_HIT_TIME_CYCLES = 10
TICK_PER_CYCLE = (1 / (CLK_FREQ_GHZ * 1e9)) * 1e12

In [7]:
df['l1dMisses'] = df['l1dAccesses'] - df['l1dHits']

df['l1iHitRate'] = 1 - np.divide(df['l1iMisses'], df['l1iAccesses'], out=np.zeros_like(df['l1iMisses'], dtype=float), where=df['l1iAccesses']!=0)
df['l1dHitRate'] = 1 - np.divide(df['l1dMisses'], df['l1dAccesses'], out=np.zeros_like(df['l1dMisses'], dtype=float), where=df['l1dAccesses']!=0)
df['l2HitRate'] = 1 - np.divide(df['l2Misses'], df['l2Accesses'], out=np.zeros_like(df['l2Misses'], dtype=float), where=df['l2Accesses']!=0)

df['l2MPKI'] = np.divide(df['l2Misses'] * 1000, df['numInstructions'], out=np.zeros_like(df['l2Misses'], dtype=float), where=df['numInstructions']!=0)

df['branchMispredictionRate'] = np.divide(df['branchMispredicted'], df['branchLookups'], out=np.zeros_like(df['branchMispredicted'], dtype=float), where=df['branchLookups']!=0)

mem_penalty_cycles = df['memAvgLat'] / TICK_PER_CYCLE

l2_miss_rate = 1 - df['l2HitRate']
df['l2AMATCycles'] = L2_HIT_TIME_CYCLES + (l2_miss_rate * mem_penalty_cycles)

l1i_miss_rate = 1 - df['l1iHitRate']
l1d_miss_rate = 1 - df['l1dHitRate']
df['l1iAMATCycles'] = L1_HIT_TIME_CYCLES + (l1i_miss_rate * df['l2AMATCycles'])
df['l1dAMATCycles'] = L1_HIT_TIME_CYCLES + (l1d_miss_rate * df['l2AMATCycles'])

In [8]:
prefix = {
    'l1a': 'l1Associativity',
    'l2a': 'l2Associativity',
    'l1s': 'l1Size',
    'l2s': 'l2Size',
    'pred': 'branchPredictor',
    'l1rp': 'l1ReplacementPolicy',
    'l1p': 'l1Prefetcher',
    'l2rp': 'l2ReplacementPolicy',
    'l2p': 'l2Prefetcher',
    'rob': 'robSize',
    'pw': 'pipelineWidth'
}

defaults = {
    'l1Associativity': 4,
    'l2Associativity': 8,
    'l1Size': '32KiB',
    'l2Size': '256KiB',
    'branchPredictor': 'tournament',
    'l1ReplacementPolicy': 'None',
    'l1Prefetcher': 'None',
    'l2ReplacementPolicy': 'None',
    'l2Prefetcher': 'None',
    'robSize': 192,
    'pipelineWidth': 4
}

for col, default_value in defaults.items():
    df[col] = default_value

for index, row in df.iterrows():
    param_string = row['parameter']
    if param_string == 'baseline':
        continue
    try:
        pfix, value = param_string.split('_', 1)
        # Use the map to get the full column name from the pfix
        if pfix in prefix:
            column = prefix[pfix]
            df.loc[index, column] = value
    except ValueError:
        pass # Ignore cases where there's no '_'

# 5. DATA TYPES: Ensure numeric columns have the correct type (using full names)
numeric_cols = ['l1Associativity', 'l2Associativity', 'robSize', 'pipelineWidth']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col])

C:\Users\kihsuakias\AppData\Local\Temp\ipykernel_35440\2821869710.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[index, column] = value
C:\Users\kihsuakias\AppData\Local\Temp\ipykernel_35440\2821869710.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '16' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[index, column] = value
C:\Users\kihsuakias\AppData\Local\Temp\ipykernel_35440\2821869710.py:41: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '12' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[index, column] = value
C:\Users\kihsuakias\AppDat

In [9]:
columnOrder = [
    'kernel',
    'parameter',
    'pipelineWidth',
    'robSize',
    'branchPredictor',
    'l1Size',
    'l1Associativity',
    'l1ReplacementPolicy',
    'l1Prefetcher',
    'l2Size',
    'l2Associativity',
    'l2ReplacementPolicy',
    'l2Prefetcher',

    'ipc',
    'cpi',
    'simSeconds',
    'numInstructions',
    'numCycles',

    'branchLookups',
    'branchMispredicted',
    'branchMispredictionRate',

    'l1iAccesses',
    'l1iMisses',
    'l1iHitRate',
    'l1iAvgMissLatency',
    'l1iAMATCycles',
    'icacheStallCycles',
    'l1dAccesses',
    'l1dHits',
    'l1dMisses',
    'l1dHitRate',
    'l1dAMATCycles',
    'l2Accesses',
    'l2Misses',
    'l2HitRate',
    'l2AvgMissLatency',
    'l2AMATCycles',
    'l2MPKI',
    'dramReadAccesses',
    'dramWriteAccesses',
    'memAvgLat',

    'commitSquashedInsts',
    'memOrderViolations',
    'dispatchRenameStalls',
    'dispatchIssueStalls',
    'iqFullEvents',
    
    'dramTotalEnergy',
    'dramAveragePower'
]
df = df[columnOrder]

In [10]:
df

,kernel,parameter,pipelineWidth,robSize,branchPredictor,l1Size,l1Associativity,l1ReplacementPolicy,l1Prefetcher,l2Size,...,dramReadAccesses,dramWriteAccesses,memAvgLat,commitSquashedInsts,memOrderViolations,dispatchRenameStalls,dispatchIssueStalls,iqFullEvents,dramTotalEnergy,dramAveragePower
0,compute,baseline,4,192,tournament,32KiB,4,None,None,256KiB,...,3.0,0.0,20208.00,3685.0,191.0,340.0,340.0,0.0,2.140462e+09,465.753000
1,compute,l1a_2,4,192,tournament,32KiB,2,None,None,256KiB,...,3.0,0.0,20208.00,3678.0,191.0,340.0,340.0,0.0,2.140466e+09,465.753971
2,compute,l1a_8,4,192,tournament,32KiB,8,None,None,256KiB,...,3.0,0.0,20208.00,3678.0,191.0,340.0,340.0,0.0,2.140463e+09,465.753345
3,compute,l1p_stride,4,192,tournament,32KiB,4,None,stride,256KiB,...,3.0,0.0,20208.00,3685.0,191.0,340.0,340.0,0.0,2.140462e+09,465.753000
4,compute,l1p_tagged,4,192,tournament,32KiB,4,None,tagged,256KiB,...,3.0,0.0,18750.00,3685.0,191.0,340.0,340.0,0.0,2.138983e+09,465.435439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,stream,pw_12,12,192,tournament,32KiB,4,None,None,256KiB,...,393221.0,133667.0,64992.00,46425.0,1386.0,13586327.0,5727.0,0.0,1.291047e+10,625.052254
80,stream,pw_8,8,192,tournament,32KiB,4,None,None,256KiB,...,393221.0,133675.0,47708.00,52684.0,1353.0,446339.0,6355.0,0.0,1.287461e+10,625.655272
81,stream,rob_128,4,128,tournament,32KiB,4,None,None,256KiB,...,393225.0,133671.0,62278.00,32973.0,1284.0,480308.0,3891.0,0.0,1.321277e+10,620.347678
82,stream,rob_256,4,256,tournament,32KiB,4,None,None,256KiB,...,393229.0,133671.0,68944.67,32708.0,1276.0,242442.0,3818.0,0.0,1.314141e+10,621.555681


In [11]:
df.columns

Index(['kernel', 'parameter', 'pipelineWidth', 'robSize', 'branchPredictor',
       'l1Size', 'l1Associativity', 'l1ReplacementPolicy', 'l1Prefetcher',
       'l2Size', 'l2Associativity', 'l2ReplacementPolicy', 'l2Prefetcher',
       'ipc', 'cpi', 'simSeconds', 'numInstructions', 'numCycles',
       'branchLookups', 'branchMispredicted', 'branchMispredictionRate',
       'l1iAccesses', 'l1iMisses', 'l1iHitRate', 'l1iAvgMissLatency',
       'l1iAMATCycles', 'icacheStallCycles', 'l1dAccesses', 'l1dHits',
       'l1dMisses', 'l1dHitRate', 'l1dAMATCycles', 'l2Accesses', 'l2Misses',
       'l2HitRate', 'l2AvgMissLatency', 'l2AMATCycles', 'l2MPKI',
       'dramReadAccesses', 'dramWriteAccesses', 'memAvgLat',
       'commitSquashedInsts', 'memOrderViolations', 'dispatchRenameStalls',
       'dispatchIssueStalls', 'iqFullEvents', 'dramTotalEnergy',
       'dramAveragePower'],
      dtype='object')

In [13]:
csvPath = scriptDir / 'simResult.csv'
df.to_csv(csvPath, index=False)

In [14]:
jsonPath = scriptDir / 'simResult.json'
result = {}
for _, row in df.iterrows():
    kernel = row['kernel']
    parameter = row['parameter']
    datapoints = row.drop(['kernel', 'parameter']).to_dict()
    
    result.setdefault(kernel, {}) \
          .setdefault(parameter, datapoints)

with open(jsonPath, "w") as f:
    json.dump(result, f, indent=4)

In [14]:
plotDir = "figs"
os.makedirs(plotDir, exist_ok=True)

sns.set_theme(style="whitegrid", context="talk")

In [15]:
sizeOrder = ['256KiB', '1MB', '4MB']
df["l2Size"] = pd.Categorical(df["l2Size"], categories=sizeOrder, ordered=True)

for kernel in df["kernel"].unique():
    plt.figure(figsize=(12, 7))
    ax = sns.lineplot(
        data=df[df["kernel"] == kernel],
        x="l2Size",
        y="ipc",
        marker="o",
        linestyle="-",
        color="royalblue"
    )

    plt.xlabel("L2 Cache Size", fontsize=13)
    plt.ylabel("IPC (Instructions Per Cycle)", fontsize=13)
    plt.title(f"IPC vs. L2 Cache Size for '{kernel}' Kernel", fontsize=15, weight="bold", pad=15)
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.tight_layout()
    plt.savefig(os.path.join(plotDir, f"ipc_vs_l2size_{kernel}.png"), dpi=300)
    plt.close()

In [16]:
df['l2Prefetcher'].unique()

array(['None', 'ampm', 'signature', 'stride', 'tagged'], dtype=object)

In [17]:
for kernel in df["kernel"].unique():
    plt.figure(figsize=(12, 7))
    ax = sns.barplot(
        data=df[df["kernel"] == kernel],
        x="robSize",
        y="dispatchIssueStalls",
        color="steelblue",
        edgecolor="black",
        errorbar=None
    )

    plt.xlabel("ROB Size", fontsize=13)
    plt.ylabel("Dispatch/Issue Stall Cycles", fontsize=13)
    plt.title(f"Stall Cycles vs. ROB Size for '{kernel}' Kernel", fontsize=15, weight="bold", pad=15)
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.tight_layout()
    plt.savefig(os.path.join(plotDir, f"stalls_vs_robsize_{kernel}.png"), dpi=300)
    plt.close()

In [18]:
for kernel in df["kernel"].unique():
    plt.figure(figsize=(12, 7))
    ax = sns.barplot(
        data=df[df["kernel"] == kernel],
        x="robSize",
        y="dispatchIssueStalls",
        color="steelblue",
        edgecolor="black",
        errorbar=None
    )

    plt.xlabel("ROB Size", fontsize=12)
    plt.ylabel("Dispatch/Issue Stall Cycles", fontsize=12)
    plt.title(f"Stall Cycles vs. ROB Size for '{kernel}' Kernel",
              fontsize=14, weight="bold")

    plt.grid(axis="y", linestyle="--", alpha=0.6)
    plt.tight_layout()

    plt.savefig(os.path.join(plotDir, f"stalls_vs_robsize_{kernel}.png"), dpi=300)
    plt.close()

In [19]:
for kernel in df["kernel"].unique():
    plt.figure(figsize=(12, 7))
    
    ax = sns.lineplot(
        data=df[df["kernel"] == kernel],
        x="robSize",
        y="memAvgLat",
        color="steelblue",
        marker="o",
        linestyle="-"
    )

    plt.xlabel("ROB Size", fontsize=12)
    plt.ylabel("Average Memory Latency (cycles)", fontsize=12)
    plt.title(f"Memory Latency vs. ROB Size for '{kernel}' Kernel",
              fontsize=14, weight="bold")

    plt.grid(axis="y", linestyle="--", alpha=0.6)
    plt.tight_layout()

    plt.savefig(os.path.join(plotDir, f"memlat_vs_robsize_{kernel}.png"), dpi=300)
    plt.close()

In [20]:
for kernel in df["kernel"].unique():
    kernel_df = df[df['kernel'] == kernel]
    
    plt.figure(figsize=(12, 7))
    
    sns.scatterplot(
        data=kernel_df,
        x='dramTotalEnergy',
        y='ipc',
        hue='l2Prefetcher',
        style='l2Prefetcher',
        s=120,
        palette="deep"
    )
    
    plt.xlabel("DRAM Total Energy", fontsize=12)
    plt.ylabel("IPC (Instructions Per Cycle)", fontsize=12)
    plt.title(f"IPC vs. DRAM Energy for L2 Prefetcher '{kernel}' Kernel",
              fontsize=14, weight="bold")
    
    plt.grid(axis="y", linestyle="--", alpha=0.6)
    plt.tight_layout()
    
    filename = f'ipc_vs_energy_L2Prefetcher_{kernel}.png'
    plt.savefig(os.path.join(plotDir, filename), dpi=300, bbox_inches='tight')
    plt.close()

In [21]:
for kernel in df["kernel"].unique():
    kernel_df = df[df['kernel'] == kernel]
    
    plt.figure(figsize=(12, 7))
    
    sns.scatterplot(
        data=kernel_df,
        x='dramTotalEnergy',
        y='ipc',
        hue='l1Prefetcher',
        style='l1Prefetcher',
        s=120,
        palette="deep"
    )
    
    plt.xlabel("DRAM Total Energy", fontsize=12)
    plt.ylabel("IPC (Instructions Per Cycle)", fontsize=12)
    plt.title(f"IPC vs. DRAM Energy for L1 Prefetcher '{kernel}' Kernel",
              fontsize=14, weight="bold")
    
    plt.grid(axis="y", linestyle="--", alpha=0.6)
    plt.tight_layout()
    
    filename = f'ipc_vs_energy_L1Prefetcher_{kernel}.png'
    plt.savefig(os.path.join(plotDir, filename), dpi=300, bbox_inches='tight')
    plt.close()

In [22]:
for kernel in df["kernel"].unique():
    kernel_df = df[df["kernel"] == kernel]
    
    plt.figure(figsize=(12, 7))
    
    sns.scatterplot(
        data=kernel_df,
        x='dramTotalEnergy',
        y='ipc',
        hue='l1ReplacementPolicy',
        style='l1ReplacementPolicy',
        s=120,
        palette="deep"
    )
    
    plt.xlabel('Total DRAM Energy (pJ)', fontsize=12)
    plt.ylabel('Instructions Per Cycle (IPC)', fontsize=12)
    plt.title(f'Performance vs. Energy for L1 Replacement Policies - {kernel} Kernel',
              fontsize=14, weight="bold")
    plt.grid(axis="y", linestyle="--", alpha=0.6)
    plt.tight_layout()
    
    filename = f'ipc_vs_energy_L1ReplacementPolicy_{kernel}.png'
    plt.savefig(os.path.join(plotDir, filename), dpi=300, bbox_inches='tight')
    plt.close()

In [28]:
selected_columns = [
    'ipc', 
    'dramTotalEnergy', 
    'l2Misses', 
    'branchMispredicted'
]

g = sns.pairplot(
    data=df,
    vars=selected_columns,
    hue='l2Prefetcher',
    diag_kind='kde',
    corner=True,
    plot_kws={'alpha': 0.6, 's': 50},
    palette="deep"
)

g.fig.suptitle(
    'Relationships Between Performance, Energy, and Microarchitectural Events',
    y=1.02,
    fontsize=16,
    weight="bold"
)

g.fig.set_size_inches(16, 14)
filename = 'comprehensive_pairplot_by_l2prefetcher.png'
plt.savefig(os.path.join(plotDir, filename), dpi=300, bbox_inches='tight')
plt.close()

In [30]:
selected_columns = [
    'ipc', 
    'dramTotalEnergy', 
    'l1dMisses', 
    'branchMispredicted'
]

g = sns.pairplot(
    data=df,
    vars=selected_columns,
    hue='l1ReplacementPolicy',
    diag_kind='kde',
    corner=True,
    plot_kws={'alpha': 0.6, 's': 50},
    palette="deep"
)

g.fig.suptitle(
    'Relationships Between Performance, Energy, and Microarchitectural Events',
    y=1.02,
    fontsize=16,
    weight="bold"
)

g.fig.set_size_inches(16, 14)
filename = 'comprehensive_pairplot_by_l1replacement.png'
plt.savefig(os.path.join(plotDir, filename), dpi=300, bbox_inches='tight')
plt.close()

In [32]:
correlation_columns = [
    'ipc',
    'cpi',
    'simSeconds',
    'dramTotalEnergy',
    'l2Misses',
    'l1dMisses',
    'branchMispredicted',
    'dramReadAccesses'
]

corr_df = df[correlation_columns]
correlation_matrix = corr_df.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(
    correlation_matrix,
    annot=True,
    fmt='.2f',
    cmap='coolwarm',
    linewidths=0.5
)
plt.title('Correlation Matrix of Key Performance and Energy Metrics', fontsize=16, weight="bold")

filename = 'correlation_heatmap.png'
plt.savefig(os.path.join(plotDir, filename), dpi=300, bbox_inches='tight')
plt.close()
